In [1]:
# 텐서플로우 2.0 기반의 huggingface 라이브러리 패키지 설치
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.3MB 8.2MB/s 
     |████████████████████████████████| 901kB 42.4MB/s 
     |████████████████████████████████| 3.3MB 48.1MB/s 
     |████████████████████████████████| 1.2MB 7.3MB/s 


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import sentencepiece as spm

In [3]:
# 미리 github에 올려놓은 데이터 파일 다운로드 (review_dataset, 실제 쇼핑몰 리뷰 크롤링 dataset)
!git clone https://github.com/yunakim2/BigDataAnalasisTermProject.git
os.listdir('BigDataAnalasisTermProject/processingdata')


Cloning into 'BigDataAnalasisTermProject'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 30 (delta 9), reused 25 (delta 7), pack-reused 0
Unpacking objects: 100% (30/30), done.


['naver_shopping.csv',
 'doctorG_review.csv',
 'buds_review.csv',
 'hera_review.csv']

## **데이터 셋 train / test 데이터로 나눔**
전체 데이터 개수가 약 20만개로 너무 많은 것을 고려하여 약 3만개의 데이터를 랜덤으로 추출한뒤 **7:3** 비율로 

감성 분석 이진 분류 모델 학습 시 필요한 데이터 셋인 `train_data` 와 모델 평가 시 필요한 데이터 셋인 `test_data`로 나눔



In [4]:
data = pd.read_csv('BigDataAnalasisTermProject/processingdata/naver_shopping.csv')
print('전체 데이터 개수 : ', len(data))
total_data = data.sample(frac=0.15)
print('랜덤 추출 데이터 개수: ', len(total_data))

size = int(len(total_data) // 3)
print('train 데이터 개수 : ', size)
print('test 데이터 개수 : ', len(total_data) - size)
test = total_data[:size]
train = total_data[size:]

전체 데이터 개수 :  199908
랜덤 추출 데이터 개수:  29986
train 데이터 개수 :  9995
test 데이터 개수 :  19991


## **bert input 생성**
한글 데이터 분석하기 위해 한국어 데이터로 훈련된 koBERT 사용


https://github.com/monologg/KoBERT-NER 에서 kobert를 tokenize 할 수 있는 코드를 가져옴

In [5]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [6]:
# koBERT 토크나이즈 임포트
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

## **bertClassificationModel의 input으로 변환하기**

bertClassifiactionModel의 인풋으로 **토큰, 세그먼트, 마스크**으로 이루어져있어 test,train 문장들을 **토큰 세그먼트,마스크 인풋으로 변환하는 과정이 필요함**


### 1. tokenizer 예시

- `tokenizer.tokenize` => 문장을 토큰화
- `tokenizer.encode` => 문장을 버트 모델의 인풋 토큰값으로 변경

In [7]:
print(tokenizer.tokenize('너무 좋은 제품이예요. 또 구매하고 싶어요.'))

['▁너무', '▁좋은', '▁제품', '이', '예', '요', '.', '▁또', '▁구매', '하고', '▁싶어', '요', '.']


In [8]:
print(tokenizer.encode('너무 좋은 제품이예요. 또 구매하고 싶어요.'))

[2, 1458, 4209, 4158, 7096, 6957, 6999, 54, 1861, 1119, 7788, 3073, 6999, 54, 3]


### 1. 토큰 인풋 변환

문장 토크나이징 후 `tokenizer.encode()` 결과 값으로 문장이 `유효한 값`이면 **1**로, `유효하지 않으면` **0**으로 채워
문장 길이가 다르지만 **버트의 인풋길이를 일정하게 고정**하기 위해 버트에서 지정한 문장 길이를 초과하면 패딩값 0으로 채운다.


In [9]:
print(tokenizer.encode('너무 좋은 제품이예요. 또 구매하고 싶어요.', max_length=128, padding = False, pad_to_max_length = True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[2, 1458, 4209, 4158, 7096, 6957, 6999, 54, 1861, 1119, 7788, 3073, 6999, 54, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


### 2. 세그멘트 인풋 변환

세그멘트는 bert 모형에 들어가 bert 모형에 맞게 고차원으로 임베딩 되는 원리임

버트 모형에서 문장이 앞 문장인지, 뒷 문장인지 표현해주는 것이고 
**지금 사용하고 있는 dataset은 한문장 이므로 '0'으로 통일한다.**

최대 길이를 128로 고정했으므로 세그먼트도 `[0]*128`가 되어야한다.


In [10]:
print([0]*128)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 3. 마스크 인풋

토큰 인풋에서 **패딩이 아닌 부분**은 `1` , **패딩인 부분**은 `0`으로 두게 되는 것

In [11]:
valid_num = len(tokenizer.encode('너무 좋은 제품이예요. 또 구매하고 싶어요'))
print(valid_num * [1] + (128-valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## **버트 인풋으로 변환하는 함수 구현**

In [12]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 128 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df.iloc[i]['reviews'], truncation=True, padding='max_length', max_length=SEQ_LEN)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df.iloc[i][LABEL_COLUMN])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 128
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = 'reviews'
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = 'label'

**Train 데이터, Test 데이터 버트 인풋으로 변환**





In [13]:
train_x, train_y = load_data(train)
test_x, test_y = load_data(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 9995/9995 [00:05<00:00, 1865.94it/s]


## **BertClassificatonModel 활용하여 감성분석 모델 구현**

In [14]:
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [15]:
bert_outputs

TFBaseModelOutputWithPooling([('last_hidden_state',
                               <KerasTensor: shape=(None, 128, 768) dtype=float32 (created by layer 'tf_bert_model')>),
                              ('pooler_output',
                               <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)])

In [16]:
bert_outputs = bert_outputs[1]

In [17]:
# Rectified Adam 옵티마이저 사용
!pip install tensorflow_addons
import tensorflow_addons as tfa
# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*2, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

     |████████████████████████████████| 686kB 8.1MB/s 


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [18]:
sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [19]:
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 92186880    input_word_ids[0][0]             
                                                                 input_masks[0][0]            

In [20]:
sentiment_model.fit(train_x, train_y, epochs=2, shuffle=True, batch_size=64, validation_data=(test_x, test_y))

Epoch 1/2
313/313 [==============================] - 390s 1s/step - loss: 0.6157 - accuracy: 0.6397 - val_loss: 0.3573 - val_accuracy: 0.8630
Epoch 2/2
313/313 [==============================] - 338s 1s/step - loss: 0.3177 - accuracy: 0.8799 - val_loss: 0.3007 - val_accuracy: 0.8900


훈련 모델 예측 성능을 F1 - Score로 체크하기 위한 함수

In [21]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df.iloc[i]['reviews'], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [22]:
test_set = predict_load_data(test)
preds = sentiment_model.predict(test_set)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 9995/9995 [00:03<00:00, 2632.84it/s]


In [23]:
# 부정이면 0, 긍정이면 1 출력
preds

array([[0.7730171 ],
       [0.9628955 ],
       [0.05315975],
       ...,
       [0.07744151],
       [0.5625638 ],
       [0.7765114 ]], dtype=float32)

 F1 SCORE를 바탕으로 성능 측정  
F1 SCORE는 precision과 recall을 가중평균하여 계산합니다  
recall은 (모델이 TRUE라고 판정한 것의 숫자)/(전체 TRUE의 숫자)  
precision은 (진짜 TRUE) / (모델이 TRUE라고 판정한 것의 숫자)

In [24]:
from sklearn.metrics import classification_report
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.94      0.83      0.88      4991
           1       0.85      0.95      0.90      5004

    accuracy                           0.89      9995
   macro avg       0.90      0.89      0.89      9995
weighted avg       0.90      0.89      0.89      9995



In [26]:
import logging
tf.get_logger().setLevel(logging.ERROR)

## 실제 쇼핑몰 데이터 리뷰 분류하기

In [27]:
doctorG_data = pd.read_csv('BigDataAnalasisTermProject/processingdata/doctorG_review.csv')
buds_data = pd.read_csv('BigDataAnalasisTermProject/processingdata/buds_review.csv')
hera_data = pd.read_csv('BigDataAnalasisTermProject/processingdata/hera_review.csv')

item_title = ['doctorG', 'buds', 'hera']
item_total_data = {'doctorG' : doctorG_data, 'buds': buds_data, 'hera_data' : hera_data}

item_total_data

{'buds':       평점  ...                                                 내용
 0      4  ...  다른건 사진 설명에 첨부했으니 간략히 사용해본 후기\n\n음질 : 제가 막귀라 적당...
 1      4  ...  갤럭시 버즈를 사용해본 갤럭시 유저로서의 현실적인 리뷰\n장점.1. 버즈와같이 고무...
 2      5  ...  갤럭서 버즈의 첫 ANC 기능이 있는 제품을 기다렸는데, 타사 ANC 적용 블루투스...
 3      5  ...  1. 착용감\n후기들 보면 착용감에 대해선 호불호가 많이 갈리는 거 같아서 불안했는...
 4      5  ...  블랙 색상이다 보니 모든 구성품 검은색으로 깔맞춤 되어있어 더 보기가 좋네요. 구입...
 ...   ..  ...                                                ...
 1995   5  ...  다나와에서 검색하고 제일 싼 곳에서 저렴하게 잘샀어요! 배송도 빨랐습니당\n포장도 ...
 1996   2  ...                        귀에 잘맞지가 않네요. 스킨캡을 사야할까봐요 ㅡㅡ
 1997   5  ...                        선물했는데 맘에들어하시네요\n음질도 좋고 좋습니다
 1998   4  ...                       배송이 느렷지만 좋네요^^ 곰표케이스도 귀엽구요!!
 1999   5  ...                       적당한 가격에 음질좋네요.\n귀도 안아프고 좋아요~
 
 [2000 rows x 4 columns],
 'doctorG':       평점  ...                                                 내용
 0      5  ...  피부타입 : 유분기 많은 지성, 소량의 트러블 있음, 극도로 예민한 피부는 아님.\...
 1      5  ...  바를때는 촉촉하고 발림도

In [28]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, truncation=True, padding='max_length')
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    print('sentence : ', sentence)
    print('predict_value %f:' %predict_value)
    print('labeled : %d' %predict_answer)
    return predict_answer

In [30]:
for title, item in item_total_data.items():
  label_data = []
  print(title ,' 분류 시작 --------- ')
  for idx in range(len(item)):
    print('idx - %d'%idx)
    label_data.append(review_evaluation_predict(item.iloc[idx]['제목']))
  item['label'] = label_data

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
idx - 750
sentence :  엄마 한개 저 한개 이렇게 샀는데 저는 너무 만족인데 엄마한텐 조금 건조한가봐용 ㅠㅠ 좋긴좋은데 좀 건조하대요. 그래서 수분크림 많이쓰고 바르라고 했어여. 호수는 엄마가원래 23호
predict_value 0.951391:
labeled : 1
idx - 751
sentence :  지성피부가 아니라서 건조한 감은 좀 있지만, 매트한만큼 지속력은 좋은 것 같아요.커버력도 괜찮은 편이라 처음에 얇게 올리고 커버가 필요한 부분 위주로 레이어드해서 올리면 깨끗하게
predict_value 0.946476:
labeled : 1
idx - 752
sentence :  매트한 걸로 화장하면 다 뜨고 촉촉한 걸로 화장하면 기름 좔좔 흐르는 환장할 피부를 가졌어요. 근데 이건 안뜨고 심지어 시간 지날 수록 제 유분이랑 어우러져서 속광 느낌 ㅠㅠ 여드
predict_value 0.104082:
labeled : 0
idx - 753
sentence :  기존 블래쿠션 만족하며 사용하다가 리뉴얼된 쿠션도 써보고싶어서 구매했어요! 더 얇게 잘 밀착되는게 장점인거같아요! 워낙 지성 피부라서 그런지 마스크에 묻어나긴 하네요 ㅠ 그래도 커
predict_value 0.952710:
labeled : 1
idx - 754
sentence :  마스크에 생각보다 아주 안묻는편은 아니예요. 그래도 피부표현 너무 마음에들고 적은양으로 커버돼서 얇게 발리는 점 좋습니다. 13호 있는것도 맘에 들고요! 마스크에 아예 안묻어나진
predict_value 0.952424:
labeled : 1
idx - 755
sentence :  키트구성도 좋구 아주 작은 틴트지만 색깔 이쁘네여~~생각보다 밝네요~ 커버는 잘 되서, 아주 소량을 펴발라야겠어요 ㅎ다음엔 21c1으로 사봐냐겠네요 엄청 건조해서 각질뜨는 얼굴인데
predict_value 0.949042:
labeled : 1
idx - 756
se

In [37]:
item_total_data['buds'].astype({'label': int})
item_total_data['hera_data'].astype({'label': int})
item_total_data['doctorG'].astype({'label' : int})

,평점,날짜,제목,내용,label
0,5,21.05.21.,"피부타입 : 유분기 많은 지성, 소량의 트러블 있음, 극도로 예민한 피부는 아님.✔...","피부타입 : 유분기 많은 지성, 소량의 트러블 있음, 극도로 예민한 피부는 아님.\...",1
1,5,21.04.17.,바를때는 촉촉하고 발림도 부드럽게 발려요.처음엔 백탁인가 싶었는데 두드리면서 빠르게...,바를때는 촉촉하고 발림도 부드럽게 발려요.처음엔 백탁인가 싶었는데 두드리면서 빠르게...,1
2,5,피엠제이.,건성인 저에게는 약간 건조한 감이 없잖아 있지만 기름기없이 뽀송한 느낌이 좋습니다....,건성인 저에게는 약간 건조한 감이 없잖아 있지만 기름기없이 뽀송한 느낌이 좋습니다....,1
3,5,21.04.21.,항상 사용하는 제품이에요지성피부인데 다른 선크림은 바르고 시간이 지나면 무겁고 진득...,항상 사용하는 제품이에요\n지성피부인데 다른 선크림은 바르고 시간이 지나면 무겁고 ...,0
4,5,21.05.12.,"항상 쓰는 제품입니다.선크림은 발림성이 중요하다고 생각하는데, 부드럽게 잘 발리고 ...","항상 쓰는 제품입니다.\n선크림은 발림성이 중요하다고 생각하는데, 부드럽게 잘 발리...",1
...,...,...,...,...,...
1995,5,21.02.17.,최고예요,배송이 어~엄청 빨랐어요. 하루만에 왔네요.발림성도 좋고 사용해도 피지가 안올라와서...,1
1996,5,21.04.09.,그냥 그래용 ㅎㅎㅎ,그냥 그래용 ㅎㅎㅎ,1
1997,5,21.03.04.,좋아요,좋아요 항상 쓰는 제품이에요 심지어 배송도 빠르네요! 잘쓰겠습니다,1
1998,5,21.04.02.,남자들도 쓸수있을정도의 좋은 선크림이에요빠른배송감사합니다,남자들도 쓸수있을정도의 좋은 선크림이에요\n빠른배송감사합니다,1


In [40]:
# 분류 데이터 Colab Notebooks에 저장

from google.colab import drive

drive.mount('/content/drive')

path = 'drive/My Drive/Colab Notebooks/'
for title, item in item_total_data.items() :
  item.to_csv(path+title+'classification_data.csv', index = False)

Mounted at /content/drive
